In [ ]:
import pandas as pd
import numpy as np
import sweetviz
from tqdm import tqdm

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
pm_data_1 = pd.read_csv('cox_call_metrics_26_31_jan_byNE_target.csv', parse_dates=['time'])
# pm_data_2 = pd.read_csv('cox_call_metrics_31-06_Feb_5min_byNE_target.csv', parse_dates=['time'])

In [ ]:
selected_columns = [
    'time',
    'network_element_b_id.name',
    'common_average_connect_time',
    'common_average_disconnect_time',
    'common_call_failed_disconnect_volume',
    'common_call_failed_call_setup_volume',
    'common_call_failed_answered_call_volume',
    'common_call_ner',
    'common_call_asr'
]
pm_data_1 = pm_data_1[selected_columns]
# pm_data_2 = pm_data_2[selected_columns]


In [ ]:
pm_data_1.info()

In [ ]:
assert pm_data_1.time.nunique()/12/24==5.0 # 5 days worth of data

In [ ]:
pm_data_1['network_element_b_id.name'].nunique()

In [ ]:
pm_data = pm_data_1.set_index(['time', 'network_element_b_id.name'])
pm_data.head()

In [ ]:
# Calculate the global minimum and maximum dates across the entire DataFrame
global_min_date = pm_data.index.get_level_values(0).min()  # Adjust the level if necessary
global_max_date = pm_data.index.get_level_values(0).max()  # Adjust the level if necessary

# Create a complete DateTimeIndex based on the global min and max dates with 5-minute frequency
global_complete_index = pd.date_range(start=global_min_date, end=global_max_date, freq='5T')

In [ ]:
print(global_min_date)
print(global_max_date)

In [ ]:
# Reindex for each mo_name
reindexed_dfs = []
for mo_name in pm_data.index.get_level_values('network_element_b_id.name').unique():
    # Select data for the current mo_name
    sub_df = pm_data.xs(mo_name, level='network_element_b_id.name')

    # Reindex the subset DataFrame using the global index and restore mo_name in the index
    sub_df_reindexed = sub_df.reindex(global_complete_index)
    sub_df_reindexed['network_element_b_id.name'] = mo_name
    sub_df_reindexed.set_index('network_element_b_id.name', append=True, inplace=True)

    reindexed_dfs.append(sub_df_reindexed)

In [ ]:
# Concatenate all the reindexed DataFrames
pm_data = pd.concat(reindexed_dfs)

In [ ]:
pm_data = pm_data.reset_index()
pm_data.rename(columns = {'level_0':'time'}, inplace = True) 

In [ ]:
pm_data.info()

In [ ]:
# print(
#     pm_data.drop('time',axis=1)\
#         .groupby('network_element_b_id.name').sum()\
#             .sort_values(by='common_call_failed_answered_call_volume', ascending=False)[['common_call_failed_answered_call_volume']]\
#                 .to_string()
#         )

In [ ]:
pm_data.to_csv("cox_data_aligned.csv", index=False)

In [ ]:
report = sweetviz.analyze([pm_data, "DataFrame"])
report.show_notebook()

In [ ]:
pm_data_filtered = pm_data.loc[pm_data['network_element_b_id.name']=='DUKEVMSIP1']

In [ ]:
import matplotlib.pyplot as plt


# Metrics to be plotted
metrics = selected_columns
metrics.remove('time')
metrics.remove('network_element_b_id.name')

# Create subplots
fig, axes = plt.subplots(len(metrics), 1, figsize=(20, 20), sharex=True)

# Loop through metrics and plot
for i, metric in enumerate(metrics):
    axes[i].plot(pm_data_filtered['time'], pm_data_filtered[metric], label=f'{metric} Data')
    #axes[i].scatter(anomaly_times, pm_data[metric].iloc[original_anomalous_indexes], color='red', label='Anomaly')
    axes[i].set_title(f'{metric}',fontsize=8)
    axes[i].set_ylabel(metric ,fontsize=8)


# Label the shared x-axis
axes[-1].set_xlabel('Time')
ticks_to_use = pm_data_filtered['time'][::30]  # Choose every 10th time point
plt.xticks(ticks_to_use)
plt.xticks(rotation=45)

# Format the dates on x-axis
date_format = mdates.DateFormatter('%Y-%m-%d %H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

# Show plot
plt.show()

In [ ]:
def calculate_rate_of_non_null_over_null(df):
    # Calculate non-null counts for each group
    non_null_counts = df.groupby('network_element_b_id.name').count()
    
    # Calculate null counts for each group
    null_counts = df.groupby('network_element_b_id.name').apply(lambda x: x.isnull().sum())
    
    # Calculate the rate of non-null to null values
    rate = non_null_counts / null_counts
    
    # Replace infinite values with NaN (occurs when dividing by zero)
    rate.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return rate

rate_df = calculate_rate_of_non_null_over_null(pm_data)

In [ ]:
rate_df.sort_values(by='common_call_failed_answered_call_volume', ascending=False)

In [ ]:
#Select columns for ML training
pm_data_ml = pm_data.copy()

In [ ]:
# Create Missingness Indicators
for column in pm_data_ml.columns:
    pm_data_ml[column + '_missing'] = pm_data_ml[column].isnull().astype(int)

In [ ]:
pm_data_ml_filtered = pm_data_ml.loc[pm_data_ml['network_element_b_id.name']=='STI-AS-WEST2']

In [ ]:
# Metrics to be plotted
metrics = ['common_average_connect_time',
           'common_call_asr', 
           'common_call_asr_missing'
          ]


In [ ]:
# Create subplots
fig, axes = plt.subplots(len(metrics), 1, figsize=(20, 6), sharex=True)

# Loop through metrics and plot
for i, metric in enumerate(metrics):
    axes[i].plot(pm_data_ml_filtered['time'], pm_data_ml_filtered[metric], label=f'{metric} Data')
    #axes[i].scatter(anomaly_times, pm_data[metric].iloc[original_anomalous_indexes], color='red', label='Anomaly')
    axes[i].set_title(f'{metric}',fontsize=8)
    axes[i].set_ylabel(metric ,fontsize=8)


# Label the shared x-axis
axes[-1].set_xlabel('Time')
ticks_to_use = pm_data_ml_filtered['time'][::1000]  # Choose every 10th time point
plt.xticks(ticks_to_use)
plt.xticks(rotation=45)

# Show plot
plt.show()

In [ ]:
pm_data_ml.info()

In [ ]:
pm_data_ml = pm_data_ml.drop(['time_missing', 'network_element_b_id.name_missing'], axis=1)

In [ ]:
#Clean the data, fill nulls with zeros 
pm_data_ml.fillna(0, inplace=True)

In [ ]:
pm_data_ml.info()

In [ ]:
selected_columns = ['common_average_connect_time',
                                'common_average_disconnect_time',
                                'common_call_failed_disconnect_volume',
                                'common_call_failed_call_setup_volume',
                                'common_call_failed_answered_call_volume', 
                                'common_call_ner',
                                'common_call_asr', 
                                'common_average_connect_time_missing',
                                'common_average_disconnect_time_missing',
                                'common_call_failed_disconnect_volume_missing',
                                'common_call_failed_call_setup_volume_missing',
                                'common_call_failed_answered_call_volume_missing',
                                'common_call_ner_missing', 
                                'common_call_asr_missing']

In [ ]:
# Function to create windowed sequences
def create_windows(data, window_size, stride=1):
    X, y = [], []
    for i in range(0, len(data) - window_size, stride):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)
    
    
# Calculate overall mean and std
all_metrics_data = pm_data_ml[selected_columns].to_numpy()

overall_mean, overall_std = np.mean(all_metrics_data, axis=0), np.std(all_metrics_data, axis=0)

In [ ]:
# List to store windowed and normalized data for all cell_ids
X_list, y_list = [], []

window_size = 144

# Group data by mo_name and loop over each group
for mo_name, group in pm_data_ml.groupby('network_element_b_id.name'):
    # Assuming the DataFrame is already sorted by time, if not sort it here
    group = group.sort_values(by='time')
    
    # Select only the columns containing the metrics
    metrics_data = group[selected_columns].to_numpy()
    
    # Normalize the data using the overall mean and std
    metrics_data_normalized = (metrics_data - overall_mean) / overall_std
    
    # Create windowed sequences
    X_window, y_window = create_windows(metrics_data_normalized, window_size, stride=3)
    
    # Append to the list
    X_list.append(X_window)
    y_list.append(y_window)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Concatenate all the windowed data
X_batched = np.concatenate(X_list, axis=0)
y_batched = np.concatenate(y_list, axis=0)

# Convert to PyTorch tensor
X_batched_tensor = torch.tensor(X_batched, dtype=torch.float32).view(-1, window_size, len(selected_columns))
y_batched_tensor = torch.tensor(y_batched, dtype=torch.float32).view(-1, len(selected_columns))

if torch.cuda.is_available():
    device = torch.device("cuda")
    X_batched_tensor = X_batched_tensor.to(device)
    y_batched_tensor = y_batched_tensor.to(device)
    print("Tensors moved to GPU")
else:
    print("GPU is not available, using CPU instead")

In [ ]:
print("X_batched_tensor shape:", X_batched_tensor.shape)
print("y_batched_tensor shape:", y_batched_tensor.shape)

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset

batch_size = 32

# Create a custom Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
train_size = int(0.8 * len(X_batched_tensor))
val_size = len(X_batched_tensor) - train_size

X_train = X_batched_tensor[:train_size]
y_train = y_batched_tensor[:train_size]

X_val = X_batched_tensor[train_size:]
y_val = y_batched_tensor[train_size:]

train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define the Transformer Model
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward):
        super(TransformerTimeSeriesModel, self).__init__()
        
        self.transformer = nn.Transformer(d_model, nhead, num_layers, dim_feedforward)
        self.linear_out = nn.Linear(d_model, d_model)
    
    def forward(self, x):
        # Expected input dimension (L, N, E) - Length, Batch size, Embedding
        x = x.permute(1, 0, 2)  # permute to fit transformer's expected input shape
        x = self.transformer(x, x)
        x = self.linear_out(x)
        x = x.permute(1, 0, 2)  # permute back to original shape
        return x[:, -1, :]  # return only the last prediction for each sequence

# Model Hyperparameters
#d_model = 6  # Dimension of the input vector, we have 6 metrics
#nhead = 2  # Number of heads in the multihead attention models
#num_layers = 3  # Number of sub-encoder-layers in the transformer encoder
#dim_feedforward = 128  # Dimension of the feedforward network model
#batch_size = 64  # your batch size

# Initialize the Model, Loss, and Optimizer
#model = TransformerTimeSeriesModel(d_model, nhead, num_layers, dim_feedforward)

# Reduced model hyperparameters
d_model = len(selected_columns)
nhead = 2  # reduced from 2
num_layers = 2  # reduced from 3
dim_feedforward = 64  # reduced from 128
#batch_size = 32  # reduced from 64

# Initialize the reduced Model, Loss, and Optimizer
model = TransformerTimeSeriesModel(d_model, nhead, num_layers, dim_feedforward)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Optional: Gradient clipping
clip_value = 1.0
        
        
#Send model to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)
    print("Model moved to GPU")
else:
    print("GPU is not available, using CPU instead")

In [ ]:
# Training Loop
import torch
import copy
import os

num_epochs = 10
patience = 2
best_val_loss = float('inf')
num_epochs_no_improve = 0
best_model = None
model_save_path = 'best_model.pth'  # Define the path where you want to save the model

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (X_train_batch, y_train_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1} Training")):
        optimizer.zero_grad()
        outputs = model(X_train_batch)
        loss = criterion(outputs, y_train_batch)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_val_batch, y_val_batch in val_loader:
            val_outputs = model(X_val_batch)
            val_loss += criterion(val_outputs, y_val_batch).item()
        val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {loss.item()}, Validation Loss: {val_loss}")

    # Checkpoint and Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        num_epochs_no_improve = 0
        best_model = copy.deepcopy(model.state_dict())
        torch.save(best_model, model_save_path)  # Save the best model state to disk
    else:
        num_epochs_no_improve += 1

    if num_epochs_no_improve == patience:
        print(f"Early stopping triggered. Validation loss did not decrease for {patience} consecutive epochs.")
        break

# Load the best model for use
model.load_state_dict(torch.load(model_save_path))


In [ ]:
checkpoint = {'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss,
              'window_size': window_size,
              'overall_mean': overall_mean,
              'overall_std': overall_std,
              # ... any other data
             }

torch.save(checkpoint, 'checkpoint_ccable2_transf_v1.pth')

In [ ]:
model.eval()
all_reconstruction_errors = []

for batch_idx, (X_batch, y_batch) in enumerate(val_loader):
    with torch.no_grad():
        y_pred = model(X_batch)
        reconstruction_error = ((y_batch.cpu() - y_pred.cpu()) ** 2).mean(dim=1).numpy()
        all_reconstruction_errors.extend(reconstruction_error)

all_reconstruction_errors = np.array(all_reconstruction_errors)

In [ ]:
np.save('all_reconstruction_errors.npy', all_reconstruction_errors)

In [ ]:
all_reconstruction_errors = np.load('all_reconstruction_errors.npy')

In [ ]:
# Create bins 
bin_ranges = pd.cut(all_reconstruction_errors, bins=50)

# Count the frequency of each bin
bin_counts = bin_ranges.value_counts().sort_index()

# Create a Pandas DataFrame for better visualization
table = pd.DataFrame({
    'Bin_Range': bin_counts.index.astype(str),
    'Frequency': bin_counts.values
})

# Print the table
print(table)

In [ ]:
threshold = np.percentile(all_reconstruction_errors, 99.95)

In [ ]:
threshold

In [ ]:
pm_data_filtered = pm_data_ml.loc[pm_data_ml['network_element_b_id.name']=='FLCHVA19DS0-NGSS']

In [ ]:
new_metrics_data = pm_data_filtered[selected_columns].to_numpy()

# Using `create_windows` function to create windowed sequences
new_X_window, new_y_window = create_windows(new_metrics_data, window_size)

# Step 4: Normalize the windowed data using original mean and std
new_X_window_normalized = (new_X_window - overall_mean) / overall_std
new_y_window_normalized = (new_y_window - overall_mean) / overall_std

# Convert to PyTorch tensor
new_X_tensor = torch.tensor(new_X_window_normalized, dtype=torch.float32).view(-1, window_size, 22)
new_y_tensor = torch.tensor(new_y_window_normalized, dtype=torch.float32).view(-1, 22)

if torch.cuda.is_available():
    device = torch.device("cuda")
    new_X_tensor = new_X_tensor.to(device)
    new_y_tensor = new_y_tensor.to(device)

# Create DataLoader if needed
new_dataset = TimeSeriesDataset(new_X_tensor, new_y_tensor)
new_loader = DataLoader(new_dataset, batch_size=batch_size, shuffle=False)

model.eval()
anomalies = []
filt_reconstruction_errors = []

for batch_idx, (X_batch, y_batch) in enumerate(new_loader):  # Replace val_loader with your test_loader if using new data
    with torch.no_grad():
        y_pred = model(X_batch)
        reconstruction_error = ((y_batch.cpu() - y_pred.cpu()) ** 2).mean(dim=1).numpy()
        
        filt_reconstruction_errors.extend(reconstruction_error)
        batch_anomalies = reconstruction_error > threshold
        anomalies.extend(batch_anomalies)

filt_reconstruction_errors = np.array(filt_reconstruction_errors)
anomalies = np.array(anomalies)
num_anomalies = np.sum(anomalies)
print(f"Number of anomalies: {num_anomalies}")

In [ ]:
#Here we calculate the number of anomalies based on the pre-calculated Anomaly score (reconstruction_errors)
anomalies_o = filt_reconstruction_errors > threshold
num_anomalies_o = np.sum(anomalies_o)
print(f"Number of anomalies: {num_anomalies_o}")

In [ ]:

#%matplotlib qt
import matplotlib.pyplot as plt
from ipywidgets import *


# Prepare anomaly indices (same as previous examples)
anomalous_indexes_val = np.where(anomalies_o)[0]
original_anomalous_indexes = anomalous_indexes_val + window_size
anomaly_times = pm_data_filtered['time'].iloc[original_anomalous_indexes]

# Metrics to be plotted
metrics = selected_columns

# Create subplots
fig, axes = plt.subplots(len(metrics), 1, figsize=(15, 35), sharex=True)

# Loop through metrics and plot
for i, metric in enumerate(metrics):
    axes[i].plot(pm_data_filtered['time'], pm_data_filtered[metric], label=f'{metric} Data')
    #axes[i].scatter(anomaly_times, pm_data[metric].iloc[original_anomalous_indexes], color='red', label='Anomaly')
    axes[i].set_title(f'{metric}',fontsize=8)
    axes[i].set_ylabel(metric ,fontsize=8)
    for anomaly_time in anomaly_times:
        axes[i].axvline(x=anomaly_time, color='red', linestyle='--', linewidth=1, label='Anomaly')
    #axes[i].legend()


# Label the shared x-axis
axes[-1].set_xlabel('Time')
ticks_to_use = pm_data_filtered['time'][::20]  # Choose every 10th time point
plt.xticks(ticks_to_use)
plt.xticks(rotation=40)

# Show plot
plt.show()

In [ ]:
#Load model

checkpoint = torch.load('checkpoint_ccable2_transf_v1.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

loss = checkpoint['loss']
window_size = checkpoint['window_size']
overall_mean = checkpoint['overall_mean']
overall_std = checkpoint['overall_std']

#Send model to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)
    print("Model moved to GPU")
else:
    print("GPU is not available, using CPU instead")

In [ ]:
np.save('FLCHVA19DS0-NGSS_reconstruction_errors.npy', filt_reconstruction_errors)

In [ ]:
filt_reconstruction_errors = np.load('FLCHVA19DS0-NGSS_reconstruction_errors.npy')